# Imports

In [1]:
import pyaurorax
import datetime
import os
import pandas as pd


# Create ephemeris records to demonstrate deleting them

In [2]:
# read API key from environment vars
api_key = os.environ["AURORAX_API_KEY"]
pyaurorax.authenticate(api_key)


In [3]:
# set values
program = "notebook-program"
platform = "notebook-platform"
instrument_type = "notebook-instrument-type"
metadata = {
    "test_meta1": "testing1",
    "test_meta2": "testing2",
}
epoch = datetime.datetime(2020, 1, 1, 0, 0)
location_geo = pyaurorax.Location(lat=51.049999, lon=-114.066666)
location_gsm = pyaurorax.Location(lat=150.25, lon=-10.75)
nbtrace = pyaurorax.Location(lat=1.23, lon=45.6)
sbtrace = pyaurorax.Location(lat=7.89, lon=101.23)

# get the ephemeris source ID
source = pyaurorax.sources.get(program, platform, instrument_type)

# create Ephemeris objects
e = pyaurorax.ephemeris.Ephemeris(data_source=source,
                                epoch=epoch,
                                location_geo=location_geo,
                                location_gsm=location_gsm,
                                nbtrace=nbtrace,
                                sbtrace=sbtrace,
                                metadata=metadata)

epoch2 = datetime.datetime(2020, 1, 1, 0, 1)
metadata2 = {
    "test_meta1": "testing12",
    "test_meta2": "testing22",
}
e2 = pyaurorax.ephemeris.Ephemeris(data_source=source,
                                epoch=epoch2,
                                location_geo=location_geo,
                                location_gsm=location_gsm,
                                nbtrace=nbtrace,
                                sbtrace=sbtrace,
                                metadata=metadata2)

# set records array
records = [e, e2]

# upload records
pyaurorax.ephemeris.upload(source.identifier, records, True)



Status code 202, stream accepted


1

In [4]:
# retrieve uploaded record
s = pyaurorax.ephemeris.search(datetime.datetime(2020, 1, 1, 0, 0, 0),
                             datetime.datetime(2020, 1, 1, 23, 59, 59),
                             programs=["notebook-program"],
                             platforms=["notebook-platform"],
                             instrument_types=["notebook-instrument-type"])

# show data as a pandas dataframe
ephemeris_data = [e.__dict__ for e in s.data]
df = pd.DataFrame(ephemeris_data)
df


,data_source,epoch,location_geo,location_gsm,nbtrace,sbtrace,metadata
0,"{'data_product_metadata_schema': [],\n 'displa...",2020-01-01 00:00:00,"{'lat': 51.049999, 'lon': -114.066666}","{'lat': 29.75, 'lon': -10.75}","{'lat': 1.23, 'lon': 45.6}","{'lat': 7.89, 'lon': 101.23}","{'test_meta1': 'testing1', 'test_meta2': 'test..."
1,"{'data_product_metadata_schema': [],\n 'displa...",2020-01-01 00:01:00,"{'lat': 51.049999, 'lon': -114.066666}","{'lat': 29.75, 'lon': -10.75}","{'lat': 1.23, 'lon': 45.6}","{'lat': 7.89, 'lon': 101.23}","{'test_meta1': 'testing12', 'test_meta2': 'tes..."


# Delete ephemeris records

In [5]:
# delete records
pyaurorax.ephemeris.delete(source, datetime.datetime(
    2020, 1, 1, 0, 0, 0), datetime.datetime(2020, 1, 1, 23, 59, 59))


In [6]:
# search ephemeris again to see that they were deleted
s = pyaurorax.ephemeris.search(datetime.datetime(2020, 1, 1, 0, 0, 0),
                             datetime.datetime(2020, 1, 1, 23, 59, 59),
                             programs=["notebook-program"],
                             platforms=["notebook-platform"],
                             instrument_types=["notebook-instrument-type"])

if len(s.data) == 0:
    print("Data was deleted")
else:
    print("Data was not deleted")


Data was deleted
